# 使用 Nexus-AM 编译负载

**Build the workload using Nexus-AM**

香山是一个裸金属设备，要在上面运行程序需要操作系统提供运行时环境，然而 Linux 等操作系统过于重量级、不便于快速测试迭代。

**我们开发了一套名为 Nexus-AM 的运行时环境**

**目的**

- 生成一些无需操作系统的负载
- 为裸机（如香山）提供运行时框架

Xiangshan is a bare-metal device that requires an operating system to provide a runtime environment for running programs. However, operating systems like Linux are too **heavyweight** and inconvenient for rapid testing and iteration.

**We present a bare metal runtime environment called Nexus-AM**

**Purpose**

- Generate workloads agilely without an OS

- Provide runtime framework for bare metal machines like XiangShan


Nexus-AM 是一个裸机运行时测试生成环境。它轻量且易于使用，并且实现了基本的系统调用接口和异常处理程序。此外，它还支持多种 ISA 和配置。

Nexus-AM is a bare-metal runtime and test-generation environment. It is lightweight and easy to use, implements basic system call interfaces and exception handlers, and supports multiple ISAs and configurations. 

`am/` 下是 nexus-am 框架的源码，`apps/` 和 `tests/` 则是常用的软件负载源码，您也可以创建自己的 app 和 test。

The `am/` directory contains the Nexus-AM framework sources; `apps/` and `tests/` hold common workload sources, and you can create your own apps and tests.

In [ ]:
%%bash
cd .. && source env.sh
cd ${AM_HOME}

tree -d -L 1

echo apps: $(ls ./apps)
echo tests: $(ls ./tests)

我们以一开始运行的 Hello, XiangShan 为例，其源码位于 `apps/hello` 目录下，我们将其输出的 “Hello, XiangShan” 替换为 “Welcome to XiangShan Turtorial” 进行编译。

We start with the "Hello, XiangShan" sample (`apps/hello`). Replace "Hello, XiangShan" with "Welcome to XiangShan Tutorial" 

此处传入 ARCH=riscv64-xs 编译到 riscv64 架构的 XiangShan 目标。

Then compile with `ARCH=riscv64-xs`.

默认情况下，riscv64 架构使用 riscv64-unknown-elf- 工具链，本次 tutorial 我们使用 GNU 工具链（riscv64-linux-gnu-），故传入 LINUX_GNU_TOOLCHAIN=1。

The default riscv64 toolchain is `riscv64-unknown-elf-`, but here we use the GNU toolchain (`riscv64-linux-gnu-`), so set `LINUX_GNU_TOOLCHAIN=1`. 

参考文件：
- `am/arch/isa/riscv64.mk`
- `am/arch/riscv64-xs.mk`

Reference files:

- `am/arch/isa/riscv64.mk`
- `am/arch/riscv64-xs.mk`

In [ ]:
%%bash
cd ../ && source env.sh >/dev/null
cd $AM_HOME/apps/hello

# Use sed to replace "Hello, XiangShan" with "Welcome to XiangShan Tutorial".
sed -i 's/Hello, XiangShan/Welcome to XiangShan Turtorial/' hello.c

# compiling
make ARCH=riscv64-xs LINUX_GNU_TOOLCHAIN=1

# check output
ls -l build

编译得到以下三个文件：
- hello-riscv64-xs.bin：程序二进制镜像（ELF 文件剥离 ELF header 等元数据），供 emu 运行使用
- hello-riscv64-xs.elf：程序的 ELF 文件
- hello-riscv64-xs.txt：程序的反汇编结果，供调试时查看

After compilation, the following three files will be generated:
- hello-riscv64-xs.bin：Program binary image (The ELF header and other metadata was removed) for emu.
- hello-riscv64-xs.elf：The program's ELF file.
- hello-riscv64-xs.txt：The program’s disassembly for debugging

In [ ]:
%%bash
cd ../ && source env.sh >/dev/null
cd $NOOP_HOME

# Use emu to run workload.
$(get_asset emu-precompile/emu) -i $AM_HOME/apps/hello/build/hello-riscv64-xs.bin --no-diff 2>/dev/null

当传入的 ARCH 不支持时，make 会打印出所有支持的 ARCH，`|| true` 是为了避免返回值非 0 导致 notebook 报错，您在实际使用中只需要执行 `make ARCH=` 即可

If the ARCH you pass isn’t supported, `make` will print all supported ARCH values. The `|| true` is only to swallow the non‑zero exit code so the notebook doesn’t error out. In normal use, just run `make ARCH=`.

In [ ]:
%%bash
cd .. && source env.sh
cd ${AM_HOME}

make ARCH= || true